#  Install, Setup gdrive and imports

In [ ]:
import os
import numpy as np
import pandas as pd


In [2]:
IN_CWR_SERVER = False 
slideset = 'cav2'

# Files


In [3]:
if IN_CWR_SERVER :
  DATADIR = "//datacd31/"
else : # LOCAL
  DATADIR = "C:\\research\\cav\\datacd31\\"

#input files and folders
mvt_outdir = "C:\\research\\cav\\datamvt\\output\\" + slideset + "\\"
MVT_CSV = '12_movat_all.csv'
DAB_NUC_CSV_DIL = "22cd31_dab_nuc_dil.csv"
DAB_NUC_CSV_UDL = "22cd31_dab_nuc_udl.csv"

#output files
outdir = DATADIR + "output\\" + slideset + "\\"
FINAL_CSV_DIL = "31cd31_mvt_dilated.csv"
FINAL_CSV_UDL = "31cd31_mvt_undilated.csv"

# configs
HEALTHY = ['HC', 'US', 'MA']
DESEASED = ['DC']     
DESEASED_YR1 = ['DY'] 
COHORTS = HEALTHY + DESEASED + DESEASED_YR1


# Funtions

In [4]:
def merge(isdilated) :
    if isdilated :
        infname = f"{outdir}{DAB_NUC_CSV_DIL}.zip"
        outfname = FINAL_CSV_DIL
    else :
        infname = f"{outdir}{DAB_NUC_CSV_UDL}.zip"
        outfname = FINAL_CSV_UDL
    if os.path.exists(infname):
        print('loading ', infname)
        raw_f_df=pd.read_csv(infname)

    if os.path.exists(f"{mvt_outdir}{MVT_CSV}.zip"):
        print(f'loading {mvt_outdir}{MVT_CSV}.zip')
        raw_mvt_df=pd.read_csv(f"{mvt_outdir}{MVT_CSV}.zip")

    df = raw_f_df
    mvt_df = raw_mvt_df

    # Add movat data columns
    df = pd.merge(left=df, right=mvt_df, on=['cohort', 'fname'], how="outer" )

    # Add computed columns
    df = df.eval('stroma = collagen + blue + white_myocar + black')
    df = df.eval('tissue = stroma + myocyte')

    # Add a column to indicate control (healthy vs deceased)
    df['control'] = np.where(df.cohort.isin(HEALTHY), 'Healthy', np.where(df.cohort.isin(DESEASED), "Diseased", 'Diseased_Yr1'))

    df.reset_index(inplace=False)
    #df.to_csv(DATADIR + "TTTTTTstat_for_t-test.csv")
    print('saving', outfname)
    df.to_csv(f"{outdir}{outfname}.zip", index=False, compression=dict(method='zip', archive_name=f'{outfname}'))
    #df.to_csv(FINAL_CSV, index=False)


# Main 

## Merge CD31 with Movat for dilated and undilated

In [5]:
merge(isdilated=True)
#merge(isdilated=False)


loading  C:\research\cav\datacd31\output\cav2\22cd31_dab_nuc_dil.csv.zip
loading C:\research\cav\datamvt\output\cav2\12_movat_all.csv.zip
saving 31cd31_mvt_dilated.csv


# Working Area